In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Praktikum 4
### Klasifikasi dengan ANN
## Deskripsi
Pada praktikum kali ini, Anda diminta untuk membuat model ANN untuk mengklasifikasi potensi seorang customer akan meninggalkan perusahaan Anda atau tidak. Istirlah populer dari fenomena ini disebut sebagai 'churn'. Tingkat churn yang tinggi (chrun rate) akan berdampak tidak baik bagi perusahaan.

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

## Pra Pengolahan Data

In [5]:
dataset = pd.read_csv('/content/drive/My Drive/Semester 5/ML/Week9/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [6]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [10]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Membuat Model ANN

In [13]:
ann = tf.keras.models.Sequential()

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [15]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [16]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training Model

In [17]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [18]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 4s 4ms/step - loss: 0.5078 - accuracy: 0.7949
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4551 - accuracy: 0.7974
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4411 - accuracy: 0.8011
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4341 - accuracy: 0.8084
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4294 - accuracy: 0.8119
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4260 - accuracy: 0.8166
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4228 - accuracy: 0.8188
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4203 - accuracy: 0.8207
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4177 - accuracy: 0.8219
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4153 - accura

## Membuat Prediksi
Diberikan informasi sebagai berikut,
*   Geography: France
*   Credit Score: 600
*   Gender: Male
*   Age: 40 years old
*   Tenure: 3 years
*   Balance: $ 60000

*   Number of Products: 2
*   Does this customer have a credit card ? Yes
*   Is this customer an Active Member: Yes
*   Estimated Salary: $ 50000
Apakah customer tersebut perlu dipertahankan?

### Modelkan Data Baru dan Buat Prediksi

In [20]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 24ms/step
[[False]]


### Prediksi Dengan Data Testing

In [21]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Cek Akurasi dan Confusion Matrix

In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1518   77]
 [ 196  209]]


0.8635